### We gonna agreggate those festival files in ONE BIG dataframe :

In [51]:
import pandas as pd
import csv
import os
import re

In [9]:
FOLDER_NAME = 'RouteDesFestivalsDataRaw'

In [29]:
df = None

file_list = os.listdir(FOLDER_NAME)
file_list.remove('.DS_Store')
for csv_file in file_list:
    if (df is None):
        df = pd.read_csv(FOLDER_NAME + '/' + csv_file)
    else:
        df = df.append(pd.read_csv(FOLDER_NAME + '/' + csv_file), ignore_index=True)

In [36]:
print str(len(df['festival'].unique())) + ' festivals found :'

148 festivals found :


In [38]:
df.head(10)

,festival,location,date,artist
0,4 VENTS,Puplinge,2011-Juin-24,RAUL DE SOUZA
1,4 VENTS,Puplinge,2011-Juin-25,SOLANGE LA FRANGE / LA PATRONNE
2,4 VENTS,Puplinge,2008-Juin-27,DJ CHILL POP
3,AFRO - PFINGSTEN,Winterthur,2014-Juin-05,EARTH WIND & FIRE EXPERIENCE FEATURING AL MCKAY
4,AFRO - PFINGSTEN,Winterthur,2014-Juin-06,OPEN SEASON
5,AFRO - PFINGSTEN,Winterthur,2014-Juin-07,STEFF LA CHEFFE
6,AFRO - PFINGSTEN,Winterthur,2014-Juin-08,THE GARIFUNA COLLECTIVE
7,AFRO - PFINGSTEN,Winterthur,2014-Juin-08,LINDIGO
8,AFRO - PFINGSTEN,Winterthur,2013-Mai-17,ALPHA BLONDY
9,AFRO - PFINGSTEN,Winterthur,2013-Mai-18,MACACO


### Then, we gonna split this dataframe into several csv files. Grouped by day.

(Artists/Bands playing the same day at the same festival will be aggregated in what we'll called the 'line-up')

In [70]:
date2 = []

for date in df['date']:
    d = date
    d = d.replace('Jan.', '01')
    d = d.replace('Fev.', '02')
    d = d.replace('Mar.', '03')
    d = d.replace('Avr.', '04')
    d = d.replace('Mai', '05')
    d = d.replace('Juin', '06')
    d = d.replace('Juil.', '07')
    d = d.replace('Aout', '08')
    d = d.replace('Sep.', '09')
    d = d.replace('Oct.', '10')
    d = d.replace('Nov.', '11')
    d = d.replace('Dec.', '12')
    date2.append(d)

In [72]:
# We first replace all the months in the dates, by numbers
df['date'] = date2

In [125]:
df.head(10)

,festival,location,date,artist
0,4 VENTS,Puplinge,2011-06-24,RAUL DE SOUZA
1,4 VENTS,Puplinge,2011-06-25,SOLANGE LA FRANGE / LA PATRONNE
2,4 VENTS,Puplinge,2008-06-27,DJ CHILL POP
3,AFRO - PFINGSTEN,Winterthur,2014-06-05,EARTH WIND & FIRE EXPERIENCE FEATURING AL MCKAY
4,AFRO - PFINGSTEN,Winterthur,2014-06-06,OPEN SEASON
5,AFRO - PFINGSTEN,Winterthur,2014-06-07,STEFF LA CHEFFE
6,AFRO - PFINGSTEN,Winterthur,2014-06-08,THE GARIFUNA COLLECTIVE
7,AFRO - PFINGSTEN,Winterthur,2014-06-08,LINDIGO
8,AFRO - PFINGSTEN,Winterthur,2013-05-17,ALPHA BLONDY
9,AFRO - PFINGSTEN,Winterthur,2013-05-18,MACACO


In [126]:
OUT_FOLDER_NAME = 'RouteDesFestivalsDataAggr'

In [127]:
grouped_df = df.groupby('date')

for key, item in grouped_df:

    # Open CSV file to write:
    csvfile = open(OUT_FOLDER_NAME+'/'+str(key)+'.csv', 'w')
    fieldnames = ['location', 'event', 'date', 'artist']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    nb_events = len(item['date'].values)
    
    for i in range(nb_events):
        writer.writerow({
            'location': item['location'].values[i],
            'event': item['festival'].values[i],
            'date': item['date'].values[i],
            'artist': item['artist'].values[i]
        })

In [135]:
# What is happening with 2016-12-03.csv ???

### Now, some festivals are hosting several artists on the same day, we ought to take care of that:
### We will agreggate all artists playing the same day at the same festival at the same place, in a new attribute 'artists' that would represent the 'line-up' of this very place this very day

In [140]:
file_list = os.listdir(OUT_FOLDER_NAME)
file_list.remove('.DS_Store')

# Will contain dataframes, one for each day
dfs = []

for csv_file in file_list:    
    dfs.append(pd.read_csv(OUT_FOLDER_NAME + '/' + csv_file))

In [173]:
# Perfect example:
# BEFORE:
dfs[1650]

,location,event,date,artist
0,Geneve,FESTIVAL DE LA BATIE,2016-09-16,CAROLINE BERGVALL
1,Geneve,FESTIVAL DE LA BATIE,2016-09-16,MIET WARLOP FRUITS OF LABOR
2,Les Acacias,FESTIVAL DE LA BATIE,2016-09-16,TOSHIKI OKADA
3,Geneve,FESTIVAL DE LA BATIE,2016-09-16,ALAIN PLATEL
4,Geneve,FESTIVAL DE LA BATIE,2016-09-16,SAMBA DE LA MUERTE
5,Les Acacias,FESTIVAL DE LA BATIE,2016-09-16,FRANCOIS CHAIGNAUD


In [211]:
test = dfs[1650]

eventGroupedFest = test.groupby(['event', 'location'])

for key, item in eventGroupedFest:
    festival_name = key[0]
    location = key[1]
    date = item['date'].values[0]
    artists = item['artist'].values
    
    print location
    print festival_name
    print date
    print artists
    
    print ''

Geneve
FESTIVAL DE LA BATIE
2016-09-16
['CAROLINE BERGVALL' 'MIET WARLOP FRUITS OF LABOR' 'ALAIN PLATEL'
 'SAMBA DE LA MUERTE']

Les Acacias
FESTIVAL DE LA BATIE
2016-09-16
['TOSHIKI OKADA' 'FRANCOIS CHAIGNAUD']



In [212]:
OUT_FOLDER_NAME_2 = 'RouteDesFestivalsDataAggr2'

In [228]:
for df in dfs:
        
    eventGroupedFest = df.groupby(['event', 'location'])

    for key, item in eventGroupedFest:
        
        festival_name = key[0]
        location = key[1]
        date = item['date'].values[0]
        artists = item['artist'].values
        artists = ', '.join(artists)
        artists = '[' + artists
        artists = artists + ']'

        # Open CSV file to write:
        csvfile = open(OUT_FOLDER_NAME_2+'/'+str(date)+'.csv', 'w')
        fieldnames = ['location', 'event', 'date', 'artists']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
            
        writer.writerow({
            'location': location,
            'event': festival_name,
            'date': date,
            'artists': str(artists)
        })

[ALERTE ROUGE]
[K2R RIDDIM]
[ANDREW HILL]
[BILL FRISELL]
[NICOLAS MASSON, INSTANT COMPOSER POOL ORCHESTRA / ICP]
[ALIESKI VIANA]
[GEORGE VASSILEV]
[DANIEL MORGADE]
[SIMON GERBER, ASTONVILLA]
[TETES RAIDES, ZOE  (Chanteuse Belge)]
[STEPHANE BLOK, CHRISTOPHE MALI]
[RAPHAEL, MICHEL JONASZ (ABRAHAM)]
[TOMAS GRAND, PAULINE CROZE]
[BRANT BJORK]
[MICHEL JONASZ (ABRAHAM)]
[TIKEN JAH FAKOLY]
[THE SUBWAYS]
[LOLE]
[JULIETTE & THE LICKS]
[CHLOE SAINTE MARIE]
[KHALED]
[ANIMAL COLLECTIVE]
[LO JO]
[WILLIAM WHITE]
[SUROIT]
[SAF SAP, CHEIKH LO]
[SIMON GERBER]
[LURA]
[ROBERT CHARLEBOIS]
[MARIO BRASSARD]
[CHLOE SAINTE MARIE, YVES SAVARD, ROBERT CHARLEBOIS]
[CHARLES DUBE]
[RICHARD DESJARDINS, CATHERINE MAJOR, MARIE CHANTAL TOUPIN]
[PATATAS CHIPAS CLUB]
[YVES SAVARD]
[PIERRE LAPOINTE, LARA]
[WATER LILLY]
[GAELE]
[AMELIE VEILLE, MARIO BRASSARD]
[SOMOGO]
[SUROIT]
[APOCALYPTICA]
[HARD FI]
[MAD CADDIES]
[DAY BY DAY]
[SINNER DC, THE BOOZE (ex BOOZE BROTHERS), FANTASMAGORIA (WORLD)]
[X SENSES]
[TIGER THE LION, C

TypeError: sequence item 0: expected string, numpy.int64 found

In [229]:
file_list = os.listdir(OUT_FOLDER_NAME_2)
file_list.remove('.DS_Store')

dfs2 = []

for csv_file in file_list:    
    dfs2.append(pd.read_csv(OUT_FOLDER_NAME_2 + '/' + csv_file))
    
# Perfect example:
# AFTER:
dfs2[1650]

CParserError: Error tokenizing data. C error: EOF inside string starting at line 2